In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Read the entire dataset
df = pd.read_csv('all-data.csv', encoding='ISO-8859-1', header=None, names=['sentiment', 'text'])

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test sets to separate CSV files
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)



In [ ]:

import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import joblib
import matplotlib.pyplot as plt

# Load NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Read CSV file for training data with specified encoding
train_df = pd.read_csv('train_data.csv', encoding='ISO-8859-1', header=None, names=['sentiment', 'text'])

# Read CSV file for testing data with specified encoding
test_df = pd.read_csv('test_data.csv', encoding='ISO-8859-1', header=None, names=['sentiment', 'text'])

# Text Preprocessing
def preprocess_text(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove special characters and punctuation
        text = re.sub(r'[^\w\s]', '', text)
        return text
    else:
        return ''

# Apply preprocessing to text data
train_df['cleaned_text'] = train_df['text'].apply(preprocess_text)
test_df['cleaned_text'] = test_df['text'].apply(preprocess_text)

# Tokenization and Lemmatization
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(lemmatized_tokens)

# Apply tokenization and lemmatization
train_df['processed_text'] = train_df['cleaned_text'].apply(tokenize_and_lemmatize)
test_df['processed_text'] = test_df['cleaned_text'].apply(tokenize_and_lemmatize)

# Splitting the data
X_train, y_train = train_df['processed_text'], train_df['sentiment']
X_test, y_test = test_df['processed_text'], test_df['sentiment']

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Logistic Regression Model
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred_lr = lr_model.predict(X_test_tfidf)
f1_score_lr = f1_score(y_test, y_pred_lr, average='weighted')
print("F1 Score (Logistic Regression):", f1_score_lr)

# Print segment text along with sentiment analysis results
# print("Segment Text\t\tActual\t\tPredicted")
# for segment, actual, predicted in zip(X_test, y_test, y_pred_lr):
#     if actual == predicted:
#         print(f"\033[92m{segment[:50] + '...' if len(segment) > 50 else segment}\033[0m", "\t\t", actual, "\t\t", predicted)
#     else:
#         print(segment[:50] + '...' if len(segment) > 50 else segment, "\t\t", actual, "\t\t", predicted)

# Save the model and vectorizer
joblib.dump(lr_model, 'sentiment_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


F1 Score (Logistic Regression): 0.7153917447111109


['tfidf_vectorizer.pkl']

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score for each class
precision = precision_score(y_test, y_pred_lr, average=None, labels=['negative', 'neutral', 'positive'])
recall = recall_score(y_test, y_pred_lr, average=None, labels=['negative', 'neutral', 'positive'])
f1 = f1_score(y_test, y_pred_lr, average=None, labels=['negative', 'neutral', 'positive'])

# Calculate weighted average of precision, recall, and F1 score
weighted_precision = precision_score(y_test, y_pred_lr, average='weighted')
weighted_recall = recall_score(y_test, y_pred_lr, average='weighted')
weighted_f1 = f1_score(y_test, y_pred_lr, average='weighted')

# Print precision, recall, and F1 score for each class
print("Precision:")
for i, label in enumerate(['negative', 'neutral', 'positive']):
    print(f"{label.capitalize()}: {precision[i]}")

print("\nRecall:")
for i, label in enumerate(['negative', 'neutral', 'positive']):
    print(f"{label.capitalize()}: {recall[i]}")

print("\nF1 Score:")
for i, label in enumerate(['negative', 'neutral', 'positive']):
    print(f"{label.capitalize()}: {f1[i]}")

# Print weighted average of precision, recall, and F1 score
print("\nWeighted Average:")
print("Precision:", weighted_precision)
print("Recall:", weighted_recall)
print("F1 Score:", weighted_f1)

# Generate classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))


Accuracy: 0.7394438722966015
Precision:
Negative: 0.7741935483870968
Neutral: 0.7278820375335121
Positive: 0.7791411042944786

Recall:
Negative: 0.43636363636363634
Neutral: 0.9509632224168126
Positive: 0.43944636678200694

F1 Score:
Negative: 0.5581395348837209
Neutral: 0.8246013667425969
Positive: 0.5619469026548674

Weighted Average:
Precision: 0.7476351317150571
Recall: 0.7394438722966015
F1 Score: 0.7153917447111109

Classification Report:
              precision    recall  f1-score   support

    negative       0.77      0.44      0.56       110
     neutral       0.73      0.95      0.82       571
    positive       0.78      0.44      0.56       289
   sentiment       0.00      0.00      0.00         1

    accuracy                           0.74       971
   macro avg       0.57      0.46      0.49       971
weighted avg       0.75      0.74      0.72       971


Confusion Matrix:
[[ 48  51  11   0]
 [  3 543  25   0]
 [ 11 151 127   0]
 [  0   1   0   0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('microsoft-news_data.csv')  # Replace 'microsoft-news_data.csv' with the path to your CSV file

# Extract the 'content' column as an array
content_array = df['headlines'].values

# Display the array
# print(content_array)

import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

# Load NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Text Preprocessing functions
def clean_text(text):
    if isinstance(text, str):
        # Remove special characters and punctuation
        text = re.sub(r'[^\w\s]', '', text)
        # Convert to lowercase
        text = text.lower()
        return text
    else:
        return ''

def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Load the trained model and TF-IDF vectorizer
lr_model = joblib.load('sentiment_model.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')



# Preprocess the text segments
cleaned_segments = [clean_text(segment) for segment in content_array]
processed_segments = [remove_stopwords(segment) for segment in cleaned_segments]
lemmatized_segments = [lemmatize_text(segment) for segment in processed_segments]

# Vectorize the preprocessed text segments using the TF-IDF vectorizer
text_tfidf = tfidf_vectorizer.transform(lemmatized_segments)

# Predict the sentiment for each vectorized text segment
segment_sentiments = lr_model.predict(text_tfidf)

# Display the results
print("Segment Text\t\tPredicted Sentiment")
for segment, predicted_sentiment in zip(content_array, segment_sentiments):
    print(segment, "\t\t", predicted_sentiment)
# Count variables for positive, negative, and neutral sentiments
positive_count = 0
negative_count = 0
neutral_count = 0

# Iterate through the predicted sentiments and count occurrences
for sentiment in segment_sentiments:
    if sentiment == 'positive':
        positive_count += 1
    elif sentiment == 'negative':
        negative_count += 1
    else:  # Neutral sentiment
        neutral_count += 1

# Print the counts
print("Positive count:", positive_count)
print("Negative count:", negative_count)
print("Neutral count:", neutral_count)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Segment Text		Predicted Sentiment
Microsoft Co. (NASDAQ:MSFT) Receives Average Recommendation of “Moderate Buy” from Brokerages 		 neutral
Contrasting SilverSun Technologies (NASDAQ:SSNT) & Globant (NYSE:GLOB) 		 positive
G2 Capital Management LLC OH Cuts Stock Position in Microsoft Co. (NASDAQ:MSFT) 		 neutral
Microsoft Co. (NASDAQ:MSFT) is Clarius Group LLC’s Largest Position 		 neutral
Hartford Financial Management Inc. Has $19.03 Million Stock Position in Microsoft Co. (NASDAQ:MSFT) 		 neutral
Microsoft Co. (NASDAQ:MSFT) Shares Sold by Essex LLC 		 neutral
Meitav Investment House Ltd. Sells 69,223 Shares of Microsoft Co. (NASDAQ:MSFT) 		 neutral
PLAYSTUDIOS, Inc. (NASDAQ:MYPS) to Post FY2024 Earnings of ($0.06) Per Share, Northland Capmk Forecasts 		 neutral
What Are Stock Buybacks? The Apple-Led Strategy Is Expected To Swell To $1 Trillion By 2025 		 neutral
How Corporate America is rethinking its sustainability and diversity efforts 		 neutral
Glancy Prongay & Murray LLP Reminds 